In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import random
import numpy as np

### Import Fashion MNIST Dataset

In [2]:
# Download and load the MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform= ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

### Set Seed

In [3]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

### Dataloaders

In [4]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

### Define LeNet5 CNN architecture

In [5]:
# Define a simple CNN model
class LeNet5(nn.Module):
    def __init__(self, input_shape=(1, 28, 28), num_classes=10):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [6]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train Function

In [7]:
def train(train_loader, model, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

### Test Function

In [8]:
def test(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_test_loss, accuracy

### Commence Training

In [9]:
train(train_dataloader, model, 50)

Epoch 1/50, Train Loss: 0.6927
Epoch 2/50, Train Loss: 0.4529
Epoch 3/50, Train Loss: 0.3804
Epoch 4/50, Train Loss: 0.3424
Epoch 5/50, Train Loss: 0.3177
Epoch 6/50, Train Loss: 0.2968
Epoch 7/50, Train Loss: 0.2807
Epoch 8/50, Train Loss: 0.2650
Epoch 9/50, Train Loss: 0.2536
Epoch 10/50, Train Loss: 0.2436
Epoch 11/50, Train Loss: 0.2334
Epoch 12/50, Train Loss: 0.2235
Epoch 13/50, Train Loss: 0.2151
Epoch 14/50, Train Loss: 0.2087
Epoch 15/50, Train Loss: 0.1982
Epoch 16/50, Train Loss: 0.1930
Epoch 17/50, Train Loss: 0.1867
Epoch 18/50, Train Loss: 0.1792
Epoch 19/50, Train Loss: 0.1732
Epoch 20/50, Train Loss: 0.1674
Epoch 21/50, Train Loss: 0.1632
Epoch 22/50, Train Loss: 0.1557
Epoch 23/50, Train Loss: 0.1502
Epoch 24/50, Train Loss: 0.1467
Epoch 25/50, Train Loss: 0.1402
Epoch 26/50, Train Loss: 0.1357
Epoch 27/50, Train Loss: 0.1319
Epoch 28/50, Train Loss: 0.1255
Epoch 29/50, Train Loss: 0.1228
Epoch 30/50, Train Loss: 0.1188
Epoch 31/50, Train Loss: 0.1141
Epoch 32/50, Trai

### Test trained model on unseen data

In [10]:
avg_test_loss, acc = test(test_dataloader, model)

In [11]:
print("Accuracy on base model: " + str(acc) + "%")

Accuracy on base model: 89.48%
